## 4.10 Coding Etiquette & Excel Reporting part 1


### This script contains the following points:

#### 1. Create data to experiment on
#### 2. Create regions column
#### 3. Create high and low activity flag
#### 4. Experiment to create profile column based on multiple conditions



In [1]:
#import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
#import newest version of data
path = r'/Users/rachelallen/CareerFoundry Achievement 4/08-2022 Instacart Basket Analysis'

In [3]:
ords_prods_all = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_all.pkl'))

In [4]:
#check data
ords_prods_all.head()

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,flagged,product_id,add_to_cart_order,reordered,...,First Name,Last Name,Gender,State,Age,date_joined,n_dependants,fam_status,income,no_first_name
0,2539329.0,1.0,1.0,2.0,8.0,NaN,No Previous Order,196,1.0,0.0,...,Linda,Nguyen,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name
1,2398795.0,1.0,2.0,3.0,7.0,15.0,Previous Order,196,1.0,1.0,...,Linda,Nguyen,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name
2,473747.0,1.0,3.0,3.0,12.0,21.0,Previous Order,196,1.0,1.0,...,Linda,Nguyen,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name
3,2254736.0,1.0,4.0,4.0,7.0,29.0,Previous Order,196,1.0,1.0,...,Linda,Nguyen,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name
4,431534.0,1.0,5.0,4.0,15.0,28.0,Previous Order,196,1.0,1.0,...,Linda,Nguyen,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name


In [5]:
ords_prods_all.shape

(32435070, 37)

In [6]:
ords_prods_all.describe()

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id,prices,mean_order,max_order,mean_spending,median_days_since_prior,Age,n_dependants,income
count,3.243506e+07,3.243506e+07,3.243506e+07,3.243506e+07,3.243506e+07,3.035696e+07,3.243507e+07,3.243506e+07,3.243506e+07,3.240487e+07,3.240487e+07,3.240487e+07,3.240487e+07,3.243506e+07,3.243506e+07,3.243506e+07,3.243506e+07,3.243506e+07,3.243506e+07
mean,1.710748e+06,1.029372e+05,1.714195e+01,2.738815e+00,1.342498e+01,1.110414e+01,2.557635e+04,8.351081e+00,5.896955e-01,7.119612e+01,9.919794e+00,1.198022e+01,1.714231e+01,3.305231e+01,1.197850e+01,1.039779e+01,4.946512e+01,1.501877e+00,9.943550e+04
std,9.873006e+05,5.946651e+04,1.753496e+01,2.090051e+00,4.246357e+00,8.778941e+00,1.409657e+04,7.126656e+00,4.918889e-01,3.821138e+01,6.281486e+00,4.956554e+02,8.663542e-01,2.515463e+01,8.321715e+01,7.131238e+00,1.848555e+01,1.118869e+00,4.305540e+04
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.521575e+01,3.000000e+00,1.000000e+00,0.000000e+00,1.800000e+01,0.000000e+00,2.590300e+04
25%,8.559430e+05,5.142000e+04,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00,1.353200e+04,3.000000e+00,0.000000e+00,3.100000e+01,4.000000e+00,4.200000e+00,1.658354e+01,1.300000e+01,7.387179e+00,6.000000e+00,3.300000e+01,1.000000e+00,6.700200e+04
50%,1.711047e+06,1.026110e+05,1.100000e+01,3.000000e+00,1.300000e+01,8.000000e+00,2.525600e+04,6.000000e+00,1.000000e+00,8.300000e+01,9.000000e+00,7.400000e+00,1.722580e+01,2.600000e+01,7.824742e+00,8.000000e+00,4.900000e+01,2.000000e+00,9.661200e+04
75%,2.565514e+06,1.543910e+05,2.400000e+01,5.000000e+00,1.600000e+01,1.500000e+01,3.793500e+04,1.100000e+01,1.000000e+00,1.070000e+02,1.600000e+01,1.130000e+01,1.781140e+01,4.700000e+01,8.253968e+00,1.300000e+01,6.500000e+01,3.000000e+00,1.279080e+05
max,3.421083e+06,2.062090e+05,9.900000e+01,6.000000e+00,2.300000e+01,3.000000e+01,4.968800e+04,1.450000e+02,1.000000e+00,1.340000e+02,2.100000e+01,9.999900e+04,2.290238e+01,9.900000e+01,2.500542e+04,3.000000e+01,8.100000e+01,3.000000e+00,5.939010e+05


In [135]:
ords_prods_all.columns

Index(['order_id', 'user_id', 'order_number', 'order_day_of_week',
       'order_hour', 'days_since_prior_order', 'flagged', 'product_id',
       'add_to_cart_order', 'reordered', '_merge', 'product_name', 'aisle_id',
       'department_id', 'prices', 'Exists', 'price_range_loc', 'busiest_day',
       'busiest_days', 'busiest_period_of_day', 'mean_order', 'max_order',
       'loyalty_flag', 'mean_spending', 'spending_flag',
       'median_days_since_prior', 'frequency_flag', 'First Name', 'Last Name',
       'Gender', 'State', 'Age', 'date_joined', 'n_dependants', 'fam_status',
       'income', 'no_first_name', 'region', 'Activity Flag'],
      dtype='object')

2. Consider any security implications that might exist for this new data. You’ll need to address any PII data in the data before continuing your analysis.

The 'First Name' and 'Last Name' columns are PII and we should ask stakeholders if they can be deleted.


In [5]:
# 3. Create regions based on the state column 
ords_prods_all['State'].value_counts(dropna = False)

Pennsylvania            667738
California              660428
Rhode Island            657662
Georgia                 657092
New Mexico              655188
Arizona                 654553
North Carolina          652329
Oklahoma                652275
Alaska                  649026
Minnesota               648424
Massachusetts           647020
Wyoming                 644927
Virginia                642103
Missouri                641323
Texas                   641051
Colorado                639820
Maine                   639175
North Dakota            638997
Alabama                 638650
Louisiana               638179
Kansas                  638021
Delaware                637477
South Carolina          637423
Oregon                  636966
Arkansas                636737
New York                636707
Nevada                  636697
Montana                 635838
South Dakota            634362
Illinois                633476
Hawaii                  633446
Washington              633445
Mississi

In [7]:
#testing with a smaller subset
result = []

for value in ords_prods_all['State']:
  if value == 'Maine' or value == 'New Hampshire':
    result.append('New England')
  elif value == 'New York' or value == 'Pennsylvania':
    result.append('Middle Atlantic')
  else:
    result.append('No State')

In [8]:
ords_prods_all['region'] = result

In [ ]:
ords_prods_all.head()

In [6]:
#update with all states
#I had to run this several times to get rid of typos

result = []

for value in ords_prods_all['State']:
  if value == 'Maine' or value == 'New Hampshire' or value == 'Vermont' or value == 'Massachusetts' or value == 'Rhode Island' or value == 'Connecticut' or value == 'New York' or value == 'Pennsylvania' or value == 'New Jersey':
    result.append('Northeast')
  elif value == 'Wisconsin' or value == 'Michigan' or value == 'Illinois' or value == 'Indiana' or value == 'Ohio' or value == 'North Dakota' or value == 'South Dakota'  or value == 'Nebraska' or value == 'Kansas' or value == 'Minnesota' or value == 'Iowa' or value == 'Missouri':
    result.append('Midwest')
  elif value == 'Delaware' or value == 'Maryland' or value == 'District of Columbia' or value == 'Virginia' or value == 'West Virginia' or value == 'North Carolina' or value == 'South Carolina' or value == 'Georgia' or value == 'Florida' or value == 'Kentucky' or value == 'Mississippi' or value == 'Tennessee' or value == 'Mississippi' or value == 'Alabama' or value == 'Oklahoma'  or value == 'Texas' or value == 'Arkansas' or value == 'Louisiana':
    result.append('South')
  elif value == 'Idaho' or value == 'Montana' or value == 'Wyoming' or value == 'Nevada' or value == 'Utah' or value == 'Colorado' or value == 'Arizona' or value == 'New Mexico' or value == 'Alaska' or value == 'Washington' or value == 'California' or value == 'Hawaii' :
    result.append('West')
  else:
    result.append('No State')

In [7]:
#create new column and append information
ords_prods_all['region'] = result

In [8]:
ords_prods_all.head()

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,flagged,product_id,add_to_cart_order,reordered,...,Last Name,Gender,State,Age,date_joined,n_dependants,fam_status,income,no_first_name,region
0,2539329.0,1.0,1.0,2.0,8.0,NaN,No Previous Order,196,1.0,0.0,...,Nguyen,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South
1,2398795.0,1.0,2.0,3.0,7.0,15.0,Previous Order,196,1.0,1.0,...,Nguyen,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South
2,473747.0,1.0,3.0,3.0,12.0,21.0,Previous Order,196,1.0,1.0,...,Nguyen,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South
3,2254736.0,1.0,4.0,4.0,7.0,29.0,Previous Order,196,1.0,1.0,...,Nguyen,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South
4,431534.0,1.0,5.0,4.0,15.0,28.0,Previous Order,196,1.0,1.0,...,Nguyen,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South


In [9]:
#make sure new column matches old column values
ords_prods_all['region'].value_counts(dropna = False)

South        10801982
West          7663651
Midwest       7604016
Northeast     5728444
No State       636977
Name: region, dtype: int64

In [10]:
#nope! That's way too many missing states let's recheck that code
check_df = ords_prods_all[['State', 'region']]

In [9]:
#try a'pivot table'
crosstab = pd.crosstab(check_df['State'], check_df['region'], dropna = False)

NameError: name 'check_df' is not defined

In [ ]:
crosstab

In [11]:
#I missed Oregon! Okay, let's add in Oregon and go back

result = []

for value in ords_prods_all['State']:
  if value == 'Maine' or value == 'New Hampshire' or value == 'Vermont' or value == 'Massachusetts' or value == 'Rhode Island' or value == 'Connecticut' or value == 'New York' or value == 'Pennsylvania' or value == 'New Jersey':
    result.append('Northeast')
  elif value == 'Wisconsin' or value == 'Michigan' or value == 'Illinois' or value == 'Indiana' or value == 'Ohio' or value == 'North Dakota' or value == 'South Dakota'  or value == 'Nebraska' or value == 'Kansas' or value == 'Minnesota' or value == 'Iowa' or value == 'Missouri':
    result.append('Midwest')
  elif value == 'Delaware' or value == 'Maryland' or value == 'District of Columbia' or value == 'Virginia' or value == 'West Virginia' or value == 'North Carolina' or value == 'South Carolina' or value == 'Georgia' or value == 'Florida' or value == 'Kentucky' or value == 'Mississippi' or value == 'Tennessee' or value == 'Mississippi' or value == 'Alabama' or value == 'Oklahoma'  or value == 'Texas' or value == 'Arkansas' or value == 'Louisiana':
    result.append('South')
  elif value == 'Idaho' or value == 'Montana' or value == 'Wyoming' or value == 'Nevada' or value == 'Utah' or value == 'Colorado' or value == 'Arizona' or value == 'New Mexico' or value == 'Alaska' or value == 'Washington' or value == 'California' or value == 'Hawaii' or value == 'Oregon' :
    result.append('West')
  else:
    result.append('No State')

In [12]:
#append information
ords_prods_all['region'] = result

In [13]:
#make sure new column matches old column values
ords_prods_all['region'].value_counts(dropna = False)

South        10801982
West          8300617
Midwest       7604016
Northeast     5728444
No State           11
Name: region, dtype: int64

In [14]:
# 3 continued: check the spending flag and the region

crosstab2 = pd.crosstab(ords_prods_all['spending_flag'], ords_prods_all['region'], dropna = False)

In [15]:
crosstab2

region,Midwest,No State,Northeast,South,West
spending_flag,,,,,
High spender,156084,0,108276,209878,160440
Low spender,7447932,0,5620168,10592104,8140177


In [ ]:
#copy to excel
crosstab2.to_clipboard()

In [ ]:
# 4 create exclusion flag for customers with < 5 orders

ords_prods_all['order_number'].value_counts(dropna = False)

In [10]:
result = []

for value in ords_prods_all['order_number']:
  if value <= 5:
    result.append('Low Activity')
  else:
    result.append('High Activity')

In [11]:
ords_prods_all['Activity Flag'] = result

In [12]:
ords_prods_all['Activity Flag'].value_counts(dropna = False)

High Activity    22809022
Low Activity      9626048
Name: Activity Flag, dtype: int64

In [13]:
#exclude low activity customers from df

ords_prods_all_high_activity = ords_prods_all[ords_prods_all['Activity Flag'] == 'High Activity']

In [14]:
ords_prods_all_high_activity.head()

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,flagged,product_id,add_to_cart_order,reordered,...,Gender,State,Age,date_joined,n_dependants,fam_status,income,no_first_name,region,Activity Flag
5,3367565.0,1.0,6.0,2.0,7.0,19.0,Previous Order,196,1.0,1.0,...,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,No State,High Activity
6,550135.0,1.0,7.0,1.0,9.0,20.0,Previous Order,196,1.0,1.0,...,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,No State,High Activity
7,3108588.0,1.0,8.0,1.0,14.0,14.0,Previous Order,196,2.0,1.0,...,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,No State,High Activity
8,2295261.0,1.0,9.0,1.0,16.0,0.0,Previous Order,196,4.0,1.0,...,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,No State,High Activity
9,2550362.0,1.0,10.0,4.0,8.0,30.0,Previous Order,196,1.0,1.0,...,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,No State,High Activity


In [15]:
#check that the number of rows is the same
ords_prods_all_high_activity['Activity Flag'].value_counts(dropna = False)

High Activity    22809022
Name: Activity Flag, dtype: int64

In [ ]:
#export for the CEO as Orders_High_Activity

ords_prods_all_high_activity.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'Orders_High_Activity.pkl'))

In [ ]:
#5 Customer Profiling
##create profile based on age, income, certain goods in the “department_id” column, and number of dependents

##young and single 

df_test = ords_prods_all[:10000000]

In [92]:
df_test.head()

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,flagged,product_id,add_to_cart_order,reordered,...,State,Age,date_joined,n_dependants,fam_status,income,no_first_name,region,Activity Flag,profile
0,2539329.0,1.0,1.0,2.0,8.0,NaN,No Previous Order,196,1.0,0.0,...,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,young and single
1,2398795.0,1.0,2.0,3.0,7.0,15.0,Previous Order,196,1.0,1.0,...,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,young and single
2,473747.0,1.0,3.0,3.0,12.0,21.0,Previous Order,196,1.0,1.0,...,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,young and single
3,2254736.0,1.0,4.0,4.0,7.0,29.0,Previous Order,196,1.0,1.0,...,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,young and single
4,431534.0,1.0,5.0,4.0,15.0,28.0,Previous Order,196,1.0,1.0,...,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,young and single


In [98]:
df_test['profile'] = np.where(((df_test['Age'] > 30) & (df_test['fam_status'] == 'married')), 'Married', 'Not Married')


/var/folders/sg/643_lsvj2pj89g1m5qn18t800000gp/T/ipykernel_27333/3546669813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['profile'] = np.where(((df_test['Age'] > 30) & (df_test['fam_status'] == 'married')), 'Married', 'Not Married')


In [99]:
df_test.head()

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,flagged,product_id,add_to_cart_order,reordered,...,State,Age,date_joined,n_dependants,fam_status,income,no_first_name,region,Activity Flag,profile
0,2539329.0,1.0,1.0,2.0,8.0,NaN,No Previous Order,196,1.0,0.0,...,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,Married
1,2398795.0,1.0,2.0,3.0,7.0,15.0,Previous Order,196,1.0,1.0,...,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,Married
2,473747.0,1.0,3.0,3.0,12.0,21.0,Previous Order,196,1.0,1.0,...,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,Married
3,2254736.0,1.0,4.0,4.0,7.0,29.0,Previous Order,196,1.0,1.0,...,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,Married
4,431534.0,1.0,5.0,4.0,15.0,28.0,Previous Order,196,1.0,1.0,...,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,Married


In [123]:
#I have tried several million things to try and get this to work and it always throws up an error

def profile(row):

 if (row['Age'] >= 30) & (row['fam_status'] == 'married'):
    return 'Old and Married'
 elif (row['Age'] < 30) & (row['fam_status'] == 'married'):
    return 'Young and Married'
 elif (row['Age'] < 30) & (row['fam_status'] == 'single'):
    return 'Young and Single'
 else:
    return'Old and Single'



In [119]:
profile(df_test)

'Old and Single'

In [124]:
df_test['Profile'] = df_test.apply(profile, axis = 1)

/var/folders/sg/643_lsvj2pj89g1m5qn18t800000gp/T/ipykernel_27333/2012666441.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Profile'] = df_test.apply(profile, axis = 1)


In [126]:
df_test.head(50)

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,flagged,product_id,add_to_cart_order,reordered,...,Age,date_joined,n_dependants,fam_status,income,no_first_name,region,Activity Flag,profile,Profile
0,2539329.0,1.0,1.0,2.0,8.0,NaN,No Previous Order,196,1.0,0.0,...,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,Married,Old and Married
1,2398795.0,1.0,2.0,3.0,7.0,15.0,Previous Order,196,1.0,1.0,...,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,Married,Old and Married
2,473747.0,1.0,3.0,3.0,12.0,21.0,Previous Order,196,1.0,1.0,...,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,Married,Old and Married
3,2254736.0,1.0,4.0,4.0,7.0,29.0,Previous Order,196,1.0,1.0,...,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,Married,Old and Married
4,431534.0,1.0,5.0,4.0,15.0,28.0,Previous Order,196,1.0,1.0,...,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,Married,Old and Married
5,3367565.0,1.0,6.0,2.0,7.0,19.0,Previous Order,196,1.0,1.0,...,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,High Activity,Married,Old and Married
6,550135.0,1.0,7.0,1.0,9.0,20.0,Previous Order,196,1.0,1.0,...,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,High Activity,Married,Old and Married
7,3108588.0,1.0,8.0,1.0,14.0,14.0,Previous Order,196,2.0,1.0,...,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,High Activity,Married,Old and Married
8,2295261.0,1.0,9.0,1.0,16.0,0.0,Previous Order,196,4.0,1.0,...,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,High Activity,Married,Old and Married
9,2550362.0,1.0,10.0,4.0,8.0,30.0,Previous Order,196,1.0,1.0,...,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,High Activity,Married,Old and Married


In [129]:
crosstab3 = pd.crosstab(df_test['fam_status'], df_test['Profile'], dropna = False)

In [130]:
crosstab3

Profile,Old and Married,Old and Single,Young and Married,Young and Single
fam_status,,,,
divorced/widowed,0,862701,0,0
living with parents and siblings,0,465976,0,0
married,6104290,0,964661,0
single,0,1159126,0,443246


In [131]:
# I am finally getting somewhere but obviously I have not taken into account all the possible conditions

def profile2(row):

 if (row['Age'] >= 30) & (row['fam_status'] == 'married'):
    return 'Older and Married'
 elif (row['Age'] < 30) & (row['fam_status'] == 'married'):
    return 'Younger and Married'
 elif (row['Age'] < 30) & (row['fam_status'] == 'single'):
    return 'Younger and Single'
 elif (row['Age'] < 30) & (row['fam_status'] == 'living with parents and siblings'):
    return 'Younger Dependant'
 elif (row['Age'] >= 30) & (row['fam_status'] == 'living with parents and siblings'):
    return 'Older Dependant'
 else:
    return'Older and Single'


In [132]:
df_test['Profile2'] = df_test.apply(profile2, axis = 1)

/var/folders/sg/643_lsvj2pj89g1m5qn18t800000gp/T/ipykernel_27333/75384350.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Profile2'] = df_test.apply(profile2, axis = 1)


In [133]:
df_test.head(100)

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,flagged,product_id,add_to_cart_order,reordered,...,date_joined,n_dependants,fam_status,income,no_first_name,region,Activity Flag,profile,Profile,Profile2
0,2539329.0,1.0,1.0,2.0,8.0,NaN,No Previous Order,196,1.0,0.0,...,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,Married,Old and Married,Older and Married
1,2398795.0,1.0,2.0,3.0,7.0,15.0,Previous Order,196,1.0,1.0,...,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,Married,Old and Married,Older and Married
2,473747.0,1.0,3.0,3.0,12.0,21.0,Previous Order,196,1.0,1.0,...,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,Married,Old and Married,Older and Married
3,2254736.0,1.0,4.0,4.0,7.0,29.0,Previous Order,196,1.0,1.0,...,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,Married,Old and Married,Older and Married
4,431534.0,1.0,5.0,4.0,15.0,28.0,Previous Order,196,1.0,1.0,...,2/17/2019,3.0,married,40423.0,No Missing Name,South,Low Activity,Married,Old and Married,Older and Married
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3317979.0,15.0,5.0,4.0,15.0,17.0,Previous Order,14715,1.0,1.0,...,6/3/2019,0.0,divorced/widowed,54313.0,No Missing Name,Midwest,Low Activity,Not Married,Old and Single,Older and Single
96,2685110.0,15.0,7.0,1.0,11.0,17.0,Previous Order,14715,3.0,1.0,...,6/3/2019,0.0,divorced/widowed,54313.0,No Missing Name,Midwest,High Activity,Not Married,Old and Single,Older and Single
97,887727.0,15.0,9.0,2.0,13.0,7.0,Previous Order,14715,1.0,1.0,...,6/3/2019,0.0,divorced/widowed,54313.0,No Missing Name,Midwest,High Activity,Not Married,Old and Single,Older and Single
98,2600170.0,15.0,11.0,2.0,9.0,14.0,Previous Order,14715,1.0,1.0,...,6/3/2019,0.0,divorced/widowed,54313.0,No Missing Name,Midwest,High Activity,Not Married,Old and Single,Older and Single


In [134]:
#create three main conditions for the full data set: young parents, young and single, older and independent 
# young parents: Age < 35 and dep > 1 and shop dept 17 (household) or 18 (babies)
#young and single: Age < 30 and fam_status = single and shop dept 5 (alcohol)
#older and independent: Age > 30 and dep = 0 and income > median
#all others: other

ords_prods_all['income'].median()

96612.0

In [16]:
def profile3(row):

  if (row['Age'] <= 35) & (row['n_dependants'] > 1 ) & (row['department_id']==17 or row['department_id']== 18):
    return 'Young Parents'
  elif (row['Age'] < 30) & (row['fam_status'] == 'single') & (row['department_id'] == 5):
    return 'Young and Single'
  elif (row['Age'] > 30) & (row['n_dependants'] == 0) & (row['income'] > 96612.0 ):
    return 'Older and independent'
  else:
    return'Other'


In [17]:
#apply to df
ords_prods_all['Profile'] = ords_prods_all.apply(profile3, axis = 1)

In [18]:
ords_prods_all.head(100)

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,flagged,product_id,add_to_cart_order,reordered,...,Gender,State,Age,date_joined,n_dependants,fam_status,income,no_first_name,region,Profile
0,2539329.0,1.0,1.0,2.0,8.0,NaN,No Previous Order,196,1.0,0.0,...,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Other
1,2398795.0,1.0,2.0,3.0,7.0,15.0,Previous Order,196,1.0,1.0,...,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Other
2,473747.0,1.0,3.0,3.0,12.0,21.0,Previous Order,196,1.0,1.0,...,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Other
3,2254736.0,1.0,4.0,4.0,7.0,29.0,Previous Order,196,1.0,1.0,...,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Other
4,431534.0,1.0,5.0,4.0,15.0,28.0,Previous Order,196,1.0,1.0,...,Female,Alabama,31.0,2/17/2019,3.0,married,40423.0,No Missing Name,South,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3317979.0,15.0,5.0,4.0,15.0,17.0,Previous Order,14715,1.0,1.0,...,Female,Indiana,69.0,6/3/2019,0.0,divorced/widowed,54313.0,No Missing Name,Midwest,Other
96,2685110.0,15.0,7.0,1.0,11.0,17.0,Previous Order,14715,3.0,1.0,...,Female,Indiana,69.0,6/3/2019,0.0,divorced/widowed,54313.0,No Missing Name,Midwest,Other
97,887727.0,15.0,9.0,2.0,13.0,7.0,Previous Order,14715,1.0,1.0,...,Female,Indiana,69.0,6/3/2019,0.0,divorced/widowed,54313.0,No Missing Name,Midwest,Other
98,2600170.0,15.0,11.0,2.0,9.0,14.0,Previous Order,14715,1.0,1.0,...,Female,Indiana,69.0,6/3/2019,0.0,divorced/widowed,54313.0,No Missing Name,Midwest,Other


In [19]:
ords_prods_all['Profile'].value_counts(dropna = False)

Other                    28322054
Older and independent     3944157
Young Parents              162001
Young and Single             6858
Name: Profile, dtype: int64

In [22]:
# it took about 45 minutes to finish creating that column so I'm saving a copy right now

ords_prods_all.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', '4_10_flagged.pkl'))